In [21]:
import string
from collections import Counter

ALFABETO = string.ascii_uppercase
M = 26

# 📘 Ejercicio 1 — Cifrado Afín

Sea el cifrado afín definido por:

$$
E_{a,b}(x) = ax + b \pmod{26}
$$

donde el alfabeto es el inglés de 26 letras  
$\{A, B, \dots, Z\}$ con la codificación:

$$
A \rightarrow 0,\quad
B \rightarrow 1,\quad
\dots,\quad
Z \rightarrow 25
$$

## a) ¿Para qué valores de \(a\) y \(b\) es posible descifrar?

Partimos del cifrado:

$$
c \equiv ax + b \pmod{26}
$$

Para descifrar, se debe despejar $(x)$:

$$
x \equiv a^{-1}(c - b) \pmod{26}
$$

Por lo tanto, es necesario que exista el inverso multiplicativo de $a$ módulo 26.
Esto ocurre **si y solo si**:

$$
\gcd(a,26) = 1
$$

Como \($26 = 2 \cdot 13$\), los valores de $a$ coprimos con 26 en el conjunto
$\{0,1,\dots,25\}$ son:

$$
a \in \{1,3,5,7,9,11,15,17,19,21,23,25\}
$$

Para el parámetro $b$  no existe restricción adicional, ya que representa
únicamente una suma módulo 26. Por tanto:

$$
b \in \{0,1,2,\dots,25\}
$$

In [22]:
def euclid(a: int, b: int) -> int:
    while b != 0:
        a, b = b, a % b
    return abs(a)

def egcd(a: int, b: int):
    if b == 0:
        return (a, 1, 0)
    g, x1, y1 = egcd(b, a % b)
    return (g, y1, x1 - (a // b) * y1)

def inverso(a: int, m: int) -> int:
    g, x, _ = egcd(a, m)
    if g != 1:
        raise ValueError("No existe inverso")
    return x % m

In [23]:
a_validos = [a for a in range(M) if euclid(a, M) == 1]
b_validos = list(range(M))

print("Valores válidos de a:", a_validos)
print("Valores válidos de b:", b_validos)

Valores válidos de a: [1, 3, 5, 7, 9, 11, 15, 17, 19, 21, 23, 25]
Valores válidos de b: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25]


## b) ¿Cuántas claves válidas $k=(a,b)$ existen?

Del apartado anterior se tiene que:

- El número de valores válidos para $a$ es $12$.
- El número de valores posibles para $b$ es $26$.

Por lo tanto, el número total de claves válidas es:

$$
|K| = 12 \times 26 = \boxed{312}
$$

In [24]:
total_claves = len(a_validos) * len(b_validos)
print("Número total de claves válidas:", total_claves)

Número total de claves válidas: 312


## c) Criptoanálisis con clave $k=(a,20)$

Se interceptó el siguiente texto cifrado con $b=20$:
OAZUSKHFKHYCXKKGWKZUOQUFKQFACUOOYHOAQPZKOKOOYZYPYXGWKHYOKTUHCWKHFUTKZYCYQPZACUTUGWKKOZUNATURYJHNYHHKWQUHH.

### i) Determinar el valor de $a$

Se realiza un análisis de frecuencia del texto cifrado. La letra que aparece con
mayor frecuencia es **K**.

En español, la letra más frecuente suele ser **E**, por lo que se plantea la
hipótesis:

$$
E \longrightarrow K
$$

Usando la codificación $(A=0,\dots,Z=25)$:

$$
E = 4, \quad K = 10
$$

Sustituyendo en la ecuación del cifrado afín $(c \equiv am + 20 \pmod{26})$, se obtiene:

$$
10 \equiv 4a + 20 \pmod{26}
$$

Restando 20 a ambos lados:

$$
4a \equiv -10 \equiv 16 \pmod{26}
$$

Como $\gcd(4,26)=2$, se divide la congruencia entre 2:

$$
2a \equiv 8 \pmod{13}
$$

El inverso de \(2\) módulo \(13\) es \(7\), ya que:

$$
2 \cdot 7 = 14 \equiv 1 \pmod{13}
$$

Por tanto:

$$
a \equiv 8 \cdot 7 = 56 \equiv 4 \pmod{13}
$$

Esto produce dos soluciones en módulo 26:

$$
a = 4 \quad \text{o} \quad a = 17
$$

Sin embargo, el cifrado solo es descifrable si $\gcd(a,26)=1$. Como:

$$
\gcd(4,26)=2 \quad \text{y} \quad \gcd(17,26)=1
$$

se concluye que:

$$
\boxed{a = 17}
$$

In [25]:
c = (
    "OAZUSKHFKHYCXKKGWKZUOQUFKQFACUOOYHOAQPZKOKOOYZYPYXGWKHYOKTUHCWKHFUTKZYCYQPZA"
    "CUTUGWKKOZUNATURYJHNYHHKWQUHH."
)

b = 20

In [26]:
frecuencias = Counter(ch for ch in c if ch in ALFABETO)
letra_mas_frecuente, _ = frecuencias.most_common(1)[0]

print("Letra más frecuente en el cifrado:", letra_mas_frecuente)

m_E = ord('E') - ord('A')
c_top = ord(letra_mas_frecuente) - ord('A')
rhs = (c_top - b) % M

candidatos_a = [a for a in range(M) if (a * m_E) % M == rhs and euclid(a, M) == 1]
a = candidatos_a[0]

print("Valor de a:", a)

Letra más frecuente en el cifrado: K
Valor de a: 17


### ii) Descifrar el texto y recuperar el mensaje original

Usando la clave $(a,b)=(17,20)$, se obtiene el siguiente texto descifrado
(sin espacios):
SILAGENTENOCREEQUELASMATEMTICASSONSIMPLESESSOLOPORQUENOSEDANCUENTADELOCOMPLICADAQUEESLAVIDAJOHNVONNEUMANN.

In [27]:
def normalizar(texto: str) -> str:
    return texto.upper()

def cifrar_afin(texto: str, a: int, b: int) -> str:
    if euclid(a, M) != 1:
        raise ValueError("a no es invertible módulo 26")
    texto = normalizar(texto)
    res = []
    for ch in texto:
        if ch in ALFABETO:
            m = ord(ch) - ord('A')
            c = (a * m + b) % M
            res.append(chr(c + ord('A')))
        else:
            res.append(ch)
    return "".join(res)

def descifrar_afin(texto: str, a: int, b: int) -> str:
    if euclid(a, M) != 1:
        raise ValueError("a no es invertible módulo 26")
    a_inv = inverso(a, M)
    texto = normalizar(texto)
    res = []
    for ch in texto:
        if ch in ALFABETO:
            c = ord(ch) - ord('A')
            m = (a_inv * (c - b)) % M
            res.append(chr(m + ord('A')))
        else:
            res.append(ch)
    return "".join(res)

In [28]:
m_descifrado = descifrar_afin(c, a, b)
print("Mensaje descifrado sin espacios:")
print(m_descifrado)

Mensaje descifrado sin espacios:
SILAGENTENOCREEQUELASMATEMTICASSONSIMPLESESSOLOPORQUENOSEDANCUENTADELOCOMPLICADAQUEESLAVIDAJOHNVONNEUMANN.


### iii) Explicación del procedimiento utilizado

1. Se contó la frecuencia de letras del texto cifrado.
2. Se identificó la letra más frecuente.
3. Se asumió que dicha letra corresponde a la letra **E**, la más frecuente en español.
4. Se convirtieron las letras a números usando la codificación $(A=0,\ldots,Z=25)$.
5. Se planteó la ecuación del cifrado afín con $(b=20)$ y se resolvió la congruencia
   modular para hallar $a$.
6. Se descartaron los valores de $a$ que no son invertibles módulo 26.
7. Se aplicó la fórmula de descifrado carácter por carácter.

### iv) Mensaje con espacios para hacerlo legible

Agregando los espacios necesarios para una correcta lectura, el mensaje original es:

> **SI LA GENTE NO CREE QUE LAS MATEMÁTICAS SON SIMPLES ES SOLO PORQUE  
> NO SE DAN CUENTA DE LO COMPLICADA QUE ES LA VIDA.  
> JOHN VON NEUMANN.**